# Combinación Energía y Clima

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Lectura y configuración del clima

In [ ]:
weather = pd.read_csv(r'Raw_Data/Temperatura.csv', parse_dates = True)

In [ ]:
print(weather.head(10))
print('\n')
print(f'Cantidad de datos: {weather.shape[0]}')
# print(weather.horaLocalidad.dtype)

In [ ]:
weather.rename(columns = {'horaLocalidad':'datetime'}, inplace = True)

In [ ]:
# La columna 'datetima' es configurada como objeto de tiempo y sus segundos llevados a cero
weather['datetime'] = pd.to_datetime(weather['datetime'])
weather['datetime'] = weather['datetime'].dt.round('min')
weather.sort_values(by=['datetime'], axis = 0, ascending = True, inplace = True)
weather.reset_index(inplace = True, drop = True)

In [ ]:
# Función para redondear los minutos a multiplos de 15
def round_minutes(dt, resolution):
    new_minute = (dt.minute // resolution + 1) * resolution
    return dt + timedelta(minutes=new_minute - dt.minute)

In [ ]:
lista = []

for date in weather['datetime']:    
    resolusion = round_minutes(date , 15)
    # date = date.replace(resolusion)
    lista.append(resolusion)
    # print(f'{fecha} con una resolusión de 15min es redondeada a: {resolusion}')
    
# print(len(lista))
weather['new_datetime'] = lista

In [ ]:
weather.head()

In [ ]:
weather.drop(columns='datetime', inplace=True)

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
weather.drop_duplicates(subset = 'new_datetime', keep = 'last', inplace = True)

In [ ]:
weather_data = weather.set_index('new_datetime')

In [ ]:
print(weather_data.head(15))
print('\n')
print(weather_data.index.min())
print(weather_data.index.max())
# print(weather_data.index.dtype)

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather_data.index, y=weather_data['temp'],
                         mode='lines',
                         name='Clima'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

## Lectura y configuración de la energia

In [ ]:
energy = pd.read_csv(r'Raw_Data/medidor_1.csv')

In [ ]:
print(energy.head(10))
print('\n')
print(f'Cantidad de datos: {energy.shape[0]}')

In [ ]:
energy.drop(columns = 'terminal', inplace = True)

In [ ]:
energy.rename(columns = {'fechahora' : 'datetime', 'demanda_activa' : 'y[kW]' }, inplace = True)

In [ ]:
#Convierto a tipo DateTimeIndex la columna "Datetime"
energy['datetime'] = pd.to_datetime(energy['datetime'])
energy.sort_values(by=['datetime'], axis = 0, ascending = True, inplace = True)
energy.reset_index(inplace = True, drop = True)

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
energy.drop_duplicates(subset = 'datetime', keep = 'last', inplace = True)

In [ ]:
energy_data = energy.set_index('datetime')
energy_data.drop(['2017-08-18 09:15:00'], inplace = True)

In [ ]:
print(energy_data.head(15))
print('\n')
print(energy_data.index.min())
print(energy_data.index.max())

In [ ]:
energy_cut = energy_data.loc['2021-04-12 08:45:00':]

In [ ]:
print(energy_cut.head())
print('\n')
print(energy_cut.tail())
print('\n')
print(energy_cut.shape[0])
print('\n')
print(energy_cut.index.dtype)

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=energy_cut.index, y=energy_cut['y[kW]'],
                         mode='lines',
                         name='Energía'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
# ##### Grafica mensual

# #datos agrupados por mes
# groups = energy_cut['y[kW]'].groupby(pd.Grouper(freq='M'))

# #set figure and axis
# fig, axs = plt.subplots(len(groups), 1, figsize=(35,35))


# for ax, (name, group) in zip(axs, groups):
    
#     #plot the data
#     ax.plot(pd.Series(group.values))

#     ax.set_xlabel('Hour of Year')
#     ax.set_ylabel('Total Load')
#     ax.set_title(name.month)
#     plt.subplots_adjust(hspace=2.5)

## Combinando ambos datos

In [ ]:
#combine the weather and energy dataframes in new df
data_comb = energy_cut.copy()

#add the weather features
data_comb['temp'] = weather_data['temp']

data_comb.head(15)

In [ ]:
print(data_comb['temp'].isna().sum())
print(data_comb['y[kW]'].isna().sum())

In [ ]:
# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre puntos de datos existentes
# data_comb['temp'].interpolate(method='linear', inplace=True)

In [ ]:
print(f'data_comb.index.freq is set to: {data_comb.index.freq}')

In [ ]:
# Custom range
data_range = pd.date_range(start = min(data_comb.index),
                          end = max(data_comb.index),
                          freq = '15min') 
#freq = '15min' indica frecuencia por hora.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo del index (datetime)
#del dataframe original, y con el valor máximo del index. Con esto lo que obtengo es TODO EL CALENDARIO
#sin datos perdidos. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
data_range

In [ ]:
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(data_comb))}')

In [ ]:
#la diferencia entre ambos df indica la cantidad de valores perdidos en el df_original
print(data_range.difference(data_comb.index))

In [ ]:
# El siguiente comando adjunta los datos "datetime" perdidos (missing) al dataset original
# pero va a generar valores NaN para la variable Target (y[kW])
full_comb = data_comb.reindex(data_range)

# Con la interpolación se tiene un datetime (set de hora y dias) continuo
print(f'La weather_data_full.index.freq ahora es: {full_comb.index.freq}')

In [ ]:
print(data_comb['temp'].isna().sum())
print(data_comb['y[kW]'].isna().sum())

In [ ]:
# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre puntos de datos existentes
data_comb['temp'].interpolate(method='linear', inplace=True)
data_comb['y[kW]'].interpolate(method='linear', inplace=True)

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_comb.index, y=data_comb['temp'],
                         mode='lines', 
                         name='Clima',
                         line=dict(color="#00ff00")))

# adjust layout
fig.update_traces(line=dict(width=1.5))
fig.show()

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_comb.index,y=data_comb['y[kW]'],
                         mode='lines',
                         name='Energia'))

# adjust layout
fig.update_traces(line=dict(width=1.5))
fig.show()

In [ ]:
data_comb.tail(15)

## Correlación clima y energia 

In [ ]:
actual_load_correlations = data_comb.corr()['y[kW]']
actual_load_correlations.sort_values(ascending=False)

In [ ]:
# Es el valor de correlación correcto?

# CSV

In [ ]:
data_comb.to_csv('data_completa.csv', encoding='utf-8')